* 25 % carga y limpieza
    * Carga con Pandas: pd.read_csv

    * Limpieza de valores error: 
        * hay columnas que tienen un valor '?', por tanto se deben reemplazar por un valor nan.

    * Cambio de tipo de dato: .astype() a numéricos y textos, por ejemplo a categorical y carat a float32 o float16

    * Limpieza de nulos (limpiar valores NaN):
        * Nulos en columnas continuas: mediana, media
        * Nulos en columnas categóricas: moda, un valor fijo

    * Encoding: texto a numérico
        * Uso de la función get_dummies() para encoding one_hot
        * Uso de map para encoding ordinal para la columna cut: 1, 2, 3, 4

* 25 % transformaciones:
    * Uso de función apply:
        * crear una columna price_iva a partir de la columna price que muestre el precio + IVA (21%)

    * Crear una nueva columna volumen combinando: x * y * z

    * Ordenar por dos columnas con sort_values():
        * tipo de corte (cut) y precio (price)

    * Agrupaciones con groupby y visualizarla
        * Agrupar por las 3 que hay de tipo categórico calculando la media, max, min por ejemplo de alguna de las numéricas: price, carat, depth

20 % distribuciones: 

* Outliers: Visualización Q1 y Q3 y calcular límites tukey y filtrar. Sobre la columna precio.

* asimetría, curtosis y transformar datos con logaritmo o raíz cuadrada

* Discretizar la columna precio por barato, medio, caro usando la función pd.cut

30 % visualizaciones:

* Seaborn EDAS:
    * univariantes:
        * histogramas y curvas de densidad
        * boxplot
        * countplot
    * bivariantes y multivariantes
        * scatterplot con hue, con size, con style
        * Calcular correlación con Pandas y mostrarla con seaborn
        * Hacer la correlación en un gráfico de barras para la columna 'price'
        * Crear una pivot table usando como index y columns algunas variables categóricas y como values usar el price y visualizarla con heatmap de seaborn
    * Combinarlas con:
        * hue, style, size, row, col usando un relplot
        * filtro


### Librerías a utilizar

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Carga y Limpieza
### Carga csv con pandas

In [2]:
df = pd.read_csv("diamonds.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53939 non-null  float64
 1   cut      53937 non-null  object 
 2   color    53938 non-null  object 
 3   clarity  53938 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53938 non-null  float64
 6   price    53938 non-null  object 
 7   x        53938 non-null  object 
 8   y        53939 non-null  float64
 9   z        53939 non-null  float64
dtypes: float64(5), object(5)
memory usage: 4.1+ MB


### Limpieza de valores error

In [4]:
df = df.replace('?', np.nan)

### Cambio de tipo de datos

In [5]:
df["carat"] = df["carat"].astype("float32")
df["price"] = df["price"].astype("float")
df["x"] = df["x"].astype("float32")
df["cut"] = df["cut"].astype("category")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53939 non-null  float32 
 1   cut      53936 non-null  category
 2   color    53937 non-null  object  
 3   clarity  53938 non-null  object  
 4   depth    53940 non-null  float64 
 5   table    53938 non-null  float64 
 6   price    53937 non-null  float64 
 7   x        53937 non-null  float32 
 8   y        53939 non-null  float64 
 9   z        53939 non-null  float64 
dtypes: category(1), float32(2), float64(5), object(2)
memory usage: 3.3+ MB


### Limpieza NaN

In [7]:
df.count()

carat      53939
cut        53936
color      53937
clarity    53938
depth      53940
table      53938
price      53937
x          53937
y          53939
z          53939
dtype: int64

In [8]:
df.isna().sum()

carat      1
cut        4
color      3
clarity    2
depth      0
table      2
price      3
x          3
y          1
z          1
dtype: int64

In [9]:
df["carat"] = df["carat"].fillna(df["carat"].mean())
df["cut"] = df["cut"].ffill()
df["color"] = df["color"].fillna(method="ffill")
df["clarity"] = df["clarity"].fillna(method="ffill")
df["table"] = df["table"].fillna(df["table"].mean())
df["price"] = df["price"].fillna(df["price"].mean())
df["price"] = df["price"].astype("int32")
df["x"] = df["x"].fillna(method="ffill")
df["y"] = df["y"].fillna(df["y"].mean())
df["z"] = df["z"].fillna(df["z"].mean())

C:\Users\user\AppData\Local\Temp\ipykernel_9904\791768244.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["color"] = df["color"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_9904\791768244.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["clarity"] = df["clarity"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_9904\791768244.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["x"] = df["x"].fillna(method="ffill")


In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float32 
 1   cut      53940 non-null  category
 2   color    53940 non-null  object  
 3   clarity  53940 non-null  object  
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int32   
 7   x        53940 non-null  float32 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(1), float32(2), float64(4), int32(1), object(2)
memory usage: 3.1+ MB


## Encoding

* Get_dummies()

In [11]:
#

* Encodig Ordinal

In [12]:
#

### Creacion columnas

* price_iva

In [13]:
#

* volumen ( x, y, z)

In [14]:
#

### Orden de columnas


* Tipo de corte

* Precio

### Agrupaciones group_by()

* carac

In [15]:
#

* price

In [16]:
#

* depth

In [17]:
#

## Seaborn EDAS
### Univariantes

* Histograma y curva de densidad

In [18]:
#

* Boxplot

In [19]:
#

* Countplot

In [20]:
#

### Bivariantes y Multivariantes
* Scatterplot

In [21]:
# con hue


In [22]:
# con style


In [23]:
# con size

#### Correlacion
* Calculo con pandas

In [24]:
#

* Mostrar con Seaborn

In [25]:
#

* Combinarlas con filtros hue, style

In [26]:
#

## Outliers
* Visualizacion Q1 y Q3 

In [27]:
#

* Limites Tukey y filtro

In [28]:
#

## Transformacion de datos
* Asimetria

* Curtosis

* Transformacion logaritmo

* Transformacion raiz cuadrada

## Discretizar la columna precio
* barato, medio, caro